In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [21]:
# Import the "all_tables_cleaned" dataset
mls_df = pd.read_csv('Clean_csv/all_players_cleaned.csv', encoding= 'unicode_escape')
mls_df

,Unnamed: 0,Player,Club,Position,GamesPlayed,GamesStarted,MinutesPlayed,GoalsScored,Assists,Shots,ShotsOnGoal,GameWinningGoal,HomeGoals,RoadGoals,GoalsPer90Minutes,ShotConversion,GameWinningAssists,HomeAssists,RoadAssists,AssistsPer90Min,FoulsCommitted,FoulsSufferred,Offsides,YellowCards,RedCards,Year
0,3814,Diego Rossi,Los Angeles FC,F,19,19,1698,14,4,71,34,1,11,3,0.74,19.7,2,3,1,0.21,6,20,12,0,0,2020
1,3815,Gyasi Zardes,Columbus Crew,F,21,20,1717,12,4,41,19,2,8,4,0.63,29.3,1,2,2,0.21,19,27,17,1,0,2020
2,3816,Raul Ruidiaz,Seattle Sounders FC,F,17,17,1427,12,4,55,23,4,8,4,0.76,21.8,0,2,2,0.25,11,13,13,2,0,2020
3,3817,Robert Beric,Chicago FIre FC,F,23,22,1931,12,1,62,29,1,7,5,0.56,19.4,0,1,0,0.05,38,58,28,1,0,2020
4,3818,Jordan Morris,Seattle Sounders FC,F,22,18,1724,10,8,39,18,3,8,2,0.52,25.6,0,5,3,0.42,6,12,13,1,0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,4500,Sebastian Mendez,Orlando City SC,M,19,11,1087,0,0,17,3,0,0,0,0.00,0.0,0,0,0,0.00,32,9,1,4,1,2020
687,4501,Kelyn Rowe,New England Revolution,M,16,10,896,0,0,17,4,0,0,0,0.00,0.0,0,0,0,0.00,13,4,0,3,0,2020
688,4502,Siem de Jong,FC Cincinnati,M,15,8,793,0,0,18,5,0,0,0,0.00,0.0,0,0,0,0.00,7,8,1,0,0,2020
689,4503,Maxime Chanot,New York City,D,20,20,1712,0,0,19,4,0,0,0,0.00,0.0,0,0,0,0.00,18,6,1,5,1,2020


In [31]:
# drop "Player" column
mls_df = mls_df.drop(['Player'],1)
mls_df.head(2)

,Unnamed: 0,Club,Position,GamesPlayed,GamesStarted,MinutesPlayed,GoalsScored,Assists,Shots,ShotsOnGoal,GameWinningGoal,HomeGoals,RoadGoals,GoalsPer90Minutes,ShotConversion,GameWinningAssists,HomeAssists,RoadAssists,AssistsPer90Min,FoulsCommitted,FoulsSufferred,Offsides,YellowCards,RedCards,Year
0,3814,Los Angeles FC,F,19,19,1698,14,4,71,34,1,11,3,0.74,19.7,2,3,1,0.21,6,20,12,0,0,2020
1,3815,Columbus Crew,F,21,20,1717,12,4,41,19,2,8,4,0.63,29.3,1,2,2,0.21,19,27,17,1,0,2020


The *mls_df* dataframe is **not showing all the columns**, as by default it shows 20 coumns. But we need to know all the Column headers to decide on testing/training data. To display all columns we can do the following,

In [32]:
# settings to display all columns
pd.set_option("display.max_columns", None)
mls_df.head(2)

,Unnamed: 0,Club,Position,GamesPlayed,GamesStarted,MinutesPlayed,GoalsScored,Assists,Shots,ShotsOnGoal,GameWinningGoal,HomeGoals,RoadGoals,GoalsPer90Minutes,ShotConversion,GameWinningAssists,HomeAssists,RoadAssists,AssistsPer90Min,FoulsCommitted,FoulsSufferred,Offsides,YellowCards,RedCards,Year
0,3814,Los Angeles FC,F,19,19,1698,14,4,71,34,1,11,3,0.74,19.7,2,3,1,0.21,6,20,12,0,0,2020
1,3815,Columbus Crew,F,21,20,1717,12,4,41,19,2,8,4,0.63,29.3,1,2,2,0.21,19,27,17,1,0,2020


In [33]:
# Check the "mls_df" datatypes.
mls_df.dtypes

Unnamed: 0              int64
Club                   object
Position               object
GamesPlayed             int64
GamesStarted            int64
MinutesPlayed           int64
GoalsScored             int64
Assists                 int64
Shots                   int64
ShotsOnGoal             int64
GameWinningGoal         int64
HomeGoals               int64
RoadGoals               int64
GoalsPer90Minutes     float64
ShotConversion        float64
GameWinningAssists      int64
HomeAssists             int64
RoadAssists             int64
AssistsPer90Min       float64
FoulsCommitted          int64
FoulsSufferred          int64
Offsides                int64
YellowCards             int64
RedCards                int64
Year                    int64
dtype: object

It looks like there are only 3 columns with **categorical** values i.e., **"Player", "Club"** and **"Position"** columns, also having consistent data type as *object*.

We should **generate a list of categorical variable** names using Python's "df.dtypes" property. In that case, we can use our variable list to perform the **one-hot encoding** *once*, rather than for each individual variable.

In [34]:
# Generate our categorical variable list
mls_cat = mls_df.dtypes[mls_df.dtypes == "object"].index.tolist()
mls_cat

['Club', 'Position']

In [35]:
mls_df.groupby(['Club','Year']).sum()

,,Unnamed: 0,GamesPlayed,GamesStarted,MinutesPlayed,GoalsScored,Assists,Shots,ShotsOnGoal,GameWinningGoal,HomeGoals,RoadGoals,GoalsPer90Minutes,ShotConversion,GameWinningAssists,HomeAssists,RoadAssists,AssistsPer90Min,FoulsCommitted,FoulsSufferred,Offsides,YellowCards,RedCards
Club,Year,,,,,,,,,,,,,,,,,,,,,,
Atlanta United,2020,120903,330,230,20634,23,26,219,81,6,10,13,2.31,180.0,6,12,14,2.61,306,330,38,47,3
CF Montreal,2020,116417,305,236,20996,34,30,257,98,8,17,17,3.93,188.4,12,12,18,7.47,279,212,51,34,5
Chicago FIre FC,2020,107844,318,230,20646,33,31,319,111,5,23,10,2.05,164.2,5,20,11,2.40,323,275,50,38,1
Colorado Rapids,2020,111615,243,175,15632,27,32,231,90,8,12,15,2.94,190.4,7,13,19,3.55,194,177,24,30,2
Columbus Crew,2020,107964,322,230,20725,34,36,251,91,12,22,12,2.78,184.1,13,22,14,2.92,309,311,41,44,0
D.C. United,2020,95004,320,234,20843,23,17,224,73,6,11,12,2.22,230.2,3,5,12,1.64,269,277,48,42,3
FC Cincinnati,2020,115158,305,229,20611,11,6,233,65,4,4,7,0.90,69.7,2,1,5,0.75,307,300,35,39,0
FC Dallas,2020,124477,312,220,19800,28,25,278,104,9,21,7,3.71,248.8,10,19,6,2.36,255,277,28,40,1
Houston Dynamo FC,2020,113808,318,229,20622,30,29,324,117,4,14,16,3.12,125.8,6,13,16,2.45,279,246,44,52,3


Before going to encode these 2 columns using Scikit-learn's OneHotEncoder module, we need to make sure that none of the categorical variables have **more than 10 unique** values and require **bucketing**.

In [36]:
# Check the number of unique values in each column
mls_df[mls_cat].nunique()

Club        26
Position     7
dtype: int64

The *Conference* column has only one unique value, that means this column has no variation in data across the dataset. So, we may **drop** this column from *mls_cat* list or use the *Team* column only for further processing.

The *Team* column has 28 unique team / club names which we need to keep as it is, because we want to predict outcome for each individual team. So, no needs for bucketing and we're ready to use **OneHotEncoder**.

In [37]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mls_df[mls_cat]))
encode_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [38]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(mls_cat)
encode_df.head()

,Club_Atlanta United,Club_CF Montreal,Club_Chicago FIre FC,Club_Colorado Rapids,Club_Columbus Crew,Club_D.C. United,Club_FC Cincinnati,Club_FC Dallas,Club_Houston Dynamo FC,Club_Inter Miami CF,Club_LA Galaxy,Club_Los Angeles FC,Club_Minnesota United FC,Club_Nashville SC,Club_New England Revolution,Club_New York City,Club_New York City Red Bulls,Club_Orlando City SC,Club_Philadelphia Union,Club_Portland Timbers,Club_Real Salt Lake,Club_San Jose Earthquakes,Club_Seattle Sounders FC,Club_Sporting Kansas City,Club_Toronto FC,Club_Vancouver Whitecaps FC,Position_D,Position_D-M,Position_F,Position_F-M,Position_M,Position_M-D,Position_M-F
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset using *panda's* **merge** and **drop** methods.

In [39]:
# Merge OneHotEncoded features and drop the originals from the updated df
mls_df = mls_df.merge(encode_df, left_index=True, right_index=True).drop(mls_df[mls_cat],1)
mls_df.head()

,Unnamed: 0,GamesPlayed,GamesStarted,MinutesPlayed,GoalsScored,Assists,Shots,ShotsOnGoal,GameWinningGoal,HomeGoals,RoadGoals,GoalsPer90Minutes,ShotConversion,GameWinningAssists,HomeAssists,RoadAssists,AssistsPer90Min,FoulsCommitted,FoulsSufferred,Offsides,YellowCards,RedCards,Year,Club_Atlanta United,Club_CF Montreal,Club_Chicago FIre FC,Club_Colorado Rapids,Club_Columbus Crew,Club_D.C. United,Club_FC Cincinnati,Club_FC Dallas,Club_Houston Dynamo FC,Club_Inter Miami CF,Club_LA Galaxy,Club_Los Angeles FC,Club_Minnesota United FC,Club_Nashville SC,Club_New England Revolution,Club_New York City,Club_New York City Red Bulls,Club_Orlando City SC,Club_Philadelphia Union,Club_Portland Timbers,Club_Real Salt Lake,Club_San Jose Earthquakes,Club_Seattle Sounders FC,Club_Sporting Kansas City,Club_Toronto FC,Club_Vancouver Whitecaps FC,Position_D,Position_D-M,Position_F,Position_F-M,Position_M,Position_M-D,Position_M-F
0,3814,19,19,1698,14,4,71,34,1,11,3,0.74,19.7,2,3,1,0.21,6,20,12,0,0,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,3815,21,20,1717,12,4,41,19,2,8,4,0.63,29.3,1,2,2,0.21,19,27,17,1,0,2020,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3816,17,17,1427,12,4,55,23,4,8,4,0.76,21.8,0,2,2,0.25,11,13,13,2,0,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3817,23,22,1931,12,1,62,29,1,7,5,0.56,19.4,0,1,0,0.05,38,58,28,1,0,2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3818,22,18,1724,10,8,39,18,3,8,2,0.52,25.6,0,5,3,0.42,6,12,13,1,0,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


The *mls_df* dataframe is **not showing all the columns**, as by default it shows 20 coumns. But we need to know all the Column headers to decide on testing/training data. To display all columns we can do the following,

In [42]:
# settings to display all columns
pd.set_option("display.max_columns", None)
mls_df.head(2)

,Unnamed: 0,GamesPlayed,GamesStarted,MinutesPlayed,GoalsScored,Assists,Shots,ShotsOnGoal,GameWinningGoal,HomeGoals,RoadGoals,GoalsPer90Minutes,ShotConversion,GameWinningAssists,HomeAssists,RoadAssists,AssistsPer90Min,FoulsCommitted,FoulsSufferred,Offsides,YellowCards,RedCards,Year,Club_Atlanta United,Club_CF Montreal,Club_Chicago FIre FC,Club_Colorado Rapids,Club_Columbus Crew,Club_D.C. United,Club_FC Cincinnati,Club_FC Dallas,Club_Houston Dynamo FC,Club_Inter Miami CF,Club_LA Galaxy,Club_Los Angeles FC,Club_Minnesota United FC,Club_Nashville SC,Club_New England Revolution,Club_New York City,Club_New York City Red Bulls,Club_Orlando City SC,Club_Philadelphia Union,Club_Portland Timbers,Club_Real Salt Lake,Club_San Jose Earthquakes,Club_Seattle Sounders FC,Club_Sporting Kansas City,Club_Toronto FC,Club_Vancouver Whitecaps FC,Position_D,Position_D-M,Position_F,Position_F-M,Position_M,Position_M-D,Position_M-F
0,3814,19,19,1698,14,4,71,34,1,11,3,0.74,19.7,2,3,1,0.21,6,20,12,0,0,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,3815,21,20,1717,12,4,41,19,2,8,4,0.63,29.3,1,2,2,0.21,19,27,17,1,0,2020,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


We need to **split** our **training** and **testing** data *before* fitting our **StandardScaler** instance. This <u> prevents testing data from influencing the standardization </u> function.

To build our training and testing datasets, we need to separate two values:

input values (which are our *independent variables* commonly referred to as **model features or "X"**) and **target output** ( *dependent variable* commonly referred to as **target or "y"** in TensorFlow documentation).

We want to build a model that will predict whether or not a team is winning; therefore, we must separate the "Wins" columns from the rest of the input data. Also, the "Conference_Overall" column has the same input for all the data, so we can drop this column.

In [54]:
# Split our preprocessed data into our features and target arrays
y = mls_df["GameWinningGoal"].values
X = mls_df.drop(["GameWinningGoal"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Now that our training and testing data have been allocated, we're ready to build our **StandardScalerobject** and standardize the numerical features.

In [55]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Our data is now **preprocessed** via one-hot encoding and standardization. 

### Define Neural Network model

For our **input layer**, we must add the **number of input features equal to the number of variables in our feature** DataFrame.

In our **hidden layers**, we'll add **two hidden layers** with only a few neurons in each layer. To create the *second hidden layer*, we'll add another **Keras Dense class** while defining our model. All of our hidden layers will use the **relu activation** function to identify nonlinear characteristics from the input values.

In the **output layer**, we'll use the sigmoid activation function that will help us predict the probability that a team is winning or not.

In [56]:
len(X_train[0])

55

In [76]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  150
hidden_nodes_layer2 = 75
hidden_nodes_layer3 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 150)               8400      
_________________________________________________________________
dense_38 (Dense)             (None, 75)                11325     
_________________________________________________________________
dense_39 (Dense)             (None, 15)                1140      
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 16        
Total params: 20,881
Trainable params: 20,881
Non-trainable params: 0
_________________________________________________________________


In [77]:
# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
17/17 [==============================] - 1s 4ms/step - loss: 0.5090 - accuracy: 0.7606
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 0.3963 - accuracy: 0.7838
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 0.3386 - accuracy: 0.8031
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 0.3166 - accuracy: 0.8031
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 0.3021 - accuracy: 0.8263
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 0.2914 - accuracy: 0.8475
Epoch 7/100
17/17 [==============================] - 0s 4ms/step - loss: 0.2822 - accuracy: 0.8571
Epoch 8/100
17/17 [==============================] - 0s 4ms/step - loss: 0.2748 - accuracy: 0.8649
Epoch 9/100
17/17 [==============================] - 0s 4ms/step - loss: 0.2701 - accuracy: 0.8784
Epoch 10/100
17/17 [==============================] - 0s 4ms/step - loss: 0.2621 - accuracy: 0.8764
Epoch 11/

17/17 [==============================] - 0s 7ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 83/100
17/17 [==============================] - 0s 8ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 84/100
17/17 [==============================] - 0s 9ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 85/100
17/17 [==============================] - 0s 8ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 86/100
17/17 [==============================] - 0s 6ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 87/100
17/17 [==============================] - 0s 6ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 88/100
17/17 [==============================] - 0s 5ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 89/100
17/17 [==============================] - 0s 6ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 90/100
17/17 [==============================] - 0s 5ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 91/100
17/17 [==============================] - 0s 8ms/step - loss: 0.2182 - accuracy: 0.9286
Epoch 92/100
